In [3]:
import os
import math
import pprint
import pyproj
import rasterio
from pyproj import Transformer
import numpy as np
from pathlib import Path
from rasterio.mask import mask
import matplotlib.pyplot as plt
from shapely.geometry import Polygon, mapping

In [ ]:
def openRawImgPath(rawImgPath):
    ''' open rawImgPath to get raw img path in dict
    # return{
            'file1name':
                '1':
                    [filename, (x,y),(x,y),(x,y),(x,y)], ... *10
                    ...
                '115':
                    [filename, (x,y),(x,y),(x,y),(x,y)], ... *10
            'file2name':
            ...
            }
    '''
    rawImgPathDict = {}

    #loop in each file
    for file in os.listdir(rawImgPath):

        #open file
        rawImgFiles = open(rawImgPath + '/' + file,'r').read()
        #split each line
        allLines = rawImgFiles.split('\n')
        key = []
        data =[]
        #loop each line in enumerate
        for countLine, line in enumerate(allLines):

            #check if that line is key(1-115)
            try:
                int(line)
                # add to key
                key.append(line)

            #if not key >> path data
            except:

                #'filename:[x,y],[x,y],[x,y],[x,y]' >> 'filename','[x,y],[x,y],[x,y],[x,y]'
                splittedLine = line.split(':')
                
                #loop each coor >> [x y]
                coorlist = [splittedLine[0]]
                for coor in splittedLine[1].split(','):
                    #'[x y]' >> 'x y'
                    coor = coor[1:-1]
                    #'x y' >> '(x,y)'
                    coors = tuple(coor.split())
                    # append each tuple of x,y to coor list
                    coorlist.append(coors)

                #add precessed path to data
                data.append(coorlist)

        # assign key and data to dict:   key(x): data(x:x+10)
        labelDict = {}
        for count in range(len(key)):
            labelDict[key[count]] = data[count*10:count*10+10]
            
        # assign labelDict (key = 1-115) to main dict(file)
        rawImgPathDict[file[:-4]] = labelDict

    return rawImgPathDict

def getAllfilePath(filePath):
    '''
    get all of the file path in the path 
    '''
    fileList = []
    # loop into each file
    for fileName in os.listdir(filePath):
        #add original path
        fileNamePath = filePath + '/' + fileName
        #add to list
        fileList.append(fileNamePath)
    return(fileList)


In [ ]:

def ClipSaveRawdata(rawPath, outPath, coor):
    return True

def loopClipSaveRawdata(rawImgPathDict, rawImgFilePathList, outputFilePathList, clipNum):
    '''
    clip and save jpg data to outpust path
    '''
    # loop into mainFileName(key (file name)) and dataLabelDict(dict value [1-115])
    mainFlieCount = 0 
    for mainFileName, dataLabelDict in rawImgPathDict.items():

        # get raw imgfile path 
        rawImgFilePath = rawImgFilePathList[mainFlieCount]
        # get output file path
        outputFilePath = outputFilePathList[mainFlieCount]

        labelCount = 1
        # loop into dataLabel(key 1-115) and fileCoorList(list value*10 [filename, coor1 - 4])
        for dataLabel, fileCoorList in dataLabelDict.items():

            # assign output file path
            LabelOutputFilePath = outputFilePath + "/RGB_" + mainFileName + "_" + str(labelCount)

            #loop only first (clipNum) number of fileCoorList
            for clipDataCount in range(clipNum):

                # separate data >> filename, [coor]
                targetData = fileCoorList[clipDataCount]
                targetFileName = targetData[0]
                targetCoor = targetData[1:]

                # assign tarrget file path
                targetFilePath = rawImgFilePath + '/' + targetFileName + '.JPG'

                # call ClipSaveRawdata to clip and save
                ClipSaveRawdata(targetFilePath, LabelOutputFilePath, targetCoor)

            labelCount += 1
                
        mainFlieCount += 1

In [187]:
rawImgFilePathList = getAllfilePath(filePath = "D:/ice-wheat/data/RawData/MAVICK3-12M/ALL")
rawImgFilePathList = rawImgFilePathList[1:-1]

rawImgPathData = "C:/Users/pacha/Desktop/masterProj/MasterProj/rawImage/rawImgPathData"
rawImgPathDict = openRawImgPath(rawImgPathData)

outputPath = "D:/ice-wheat/data/dataForProcess/mainData/RGB"
outputPathList = getAllfilePath(outputPath)
outputPathList = outputPathList[:-1]

loopClipSaveRawdata(rawImgPathDict, rawImgFilePathList, outputPathList, 3)

D:/ice-wheat/data/RawData/MAVICK3-12M/ALL/DJI_202401181250_002_aobayamaWheat/DJI_20240118130341_0044_D.JPG
D:/ice-wheat/data/RawData/MAVICK3-12M/ALL/DJI_202401181250_002_aobayamaWheat/DJI_20240118130339_0043_D.JPG
D:/ice-wheat/data/RawData/MAVICK3-12M/ALL/DJI_202401181250_002_aobayamaWheat/DJI_20240118130343_0045_D.JPG
D:/ice-wheat/data/RawData/MAVICK3-12M/ALL/DJI_202401181250_002_aobayamaWheat/DJI_20240118130341_0044_D.JPG
D:/ice-wheat/data/RawData/MAVICK3-12M/ALL/DJI_202401181250_002_aobayamaWheat/DJI_20240118130429_0068_D.JPG
D:/ice-wheat/data/RawData/MAVICK3-12M/ALL/DJI_202401181250_002_aobayamaWheat/DJI_20240118130339_0043_D.JPG
D:/ice-wheat/data/RawData/MAVICK3-12M/ALL/DJI_202401181250_002_aobayamaWheat/DJI_20240118130429_0068_D.JPG
D:/ice-wheat/data/RawData/MAVICK3-12M/ALL/DJI_202401181250_002_aobayamaWheat/DJI_20240118130431_0069_D.JPG
D:/ice-wheat/data/RawData/MAVICK3-12M/ALL/DJI_202401181250_002_aobayamaWheat/DJI_20240118130427_0067_D.JPG
D:/ice-wheat/data/RawData/MAVICK3-12M